# Projeto 1 - Ciência dos Dados

Nome: _____

Nome: _____

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [38]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re 
from IPython.display import display

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [39]:

filename = 'Playstation5.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo Playstation5.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [40]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevância,% de 1
0,confira o novo trailer do playstation 5 https:...,0,45.333333
1,todos detalhes do playstation 5 serão revelado...,0,NaN
2,corre pra comprar o playstation 5 que tá com q...,1,NaN
3,eu aposto no dia 16 como essa sony safadona va...,1,NaN
4,rt @playstationpt: a próxima apresentação da p...,0,NaN


In [41]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevância,% de 1
0,entenda como ficará a venda de eletrodoméstico...,0,48.292683
1,o novo #playstation 5 está previsto para ser l...,0,NaN
2,"@jose_simao vem cá, o preço do novo playstatio...",1,NaN
3,@yasminxlima me dá um playstation 5,1,NaN
4,propaganda do playstation 5 aparece em partida...,0,NaN


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [42]:
#Configuração do train
train['Treinamento'] = train['Treinamento'].astype('category')
train['Relevância'] = train['Relevância'].astype('category')
train.Relevância.cat.categories = ['irrelevante', 'relevante']

#Configuração do test
test['Teste'] = test['Teste'].astype('category')
test['Relevância'] = test['Relevância'].astype('category')
test.Relevância.cat.categories = ['irrelevante', 'relevante']



In [43]:
display(test.head(3))
display(train.head(3))



,Teste,Relevância,% de 1
0,entenda como ficará a venda de eletrodoméstico...,irrelevante,48.292683
1,o novo #playstation 5 está previsto para ser l...,irrelevante,NaN
2,"@jose_simao vem cá, o preço do novo playstatio...",relevante,NaN


,Treinamento,Relevância,% de 1
0,confira o novo trailer do playstation 5 https:...,irrelevante,45.333333
1,todos detalhes do playstation 5 serão revelado...,irrelevante,NaN
2,corre pra comprar o playstation 5 que tá com q...,relevante,NaN


In [44]:
# Tabela de frequências relativas => Probabilidades marginais
p=train.Relevância.value_counts(True, sort=False).round(4)
I=p[0]
R=p[1]
print(f'Irrelevante:{I}\n  Relevante:{R}')

Irrelevante:0.5467
  Relevante:0.4533


In [45]:
def cleanup(text):
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [46]:
# Separando por relevância
#Irrelevante
tweet_i = train.loc[train.Relevância =='irrelevante','Treinamento'].str.lower()

#Relevante
tweet_r = train.loc[train.Relevância =='relevante','Treinamento'].str.lower()

#Teste
tweet_t = test.loc[:,'Teste'].str.lower()


display(tweet_i.head(3))
display(tweet_r.head(3))
display(tweet_t.head(3))

0    confira o novo trailer do playstation 5 https:...
1    todos detalhes do playstation 5 serão revelado...
4    rt @playstationpt: a próxima apresentação da p...
Name: Treinamento, dtype: object

2    corre pra comprar o playstation 5 que tá com q...
3    eu aposto no dia 16 como essa sony safadona va...
8    o torcedor gremista hoje é aquele molequinho q...
Name: Treinamento, dtype: object

0    entenda como ficará a venda de eletrodoméstico...
1    o novo #playstation 5 está previsto para ser l...
2    @jose_simao vem cá, o preço do novo playstatio...
Name: Teste, dtype: object

In [70]:
#Dados total
dados = ''
for i in range(0,train.shape[0]):    
    # Seleciona e limpa o tweet
    tweet_raw = str(train.iloc[i,0])
    tweet = cleanup(tweet_raw.lower())
    dados = dados + tweet

#Dados relevante
dados_r = ''
for i in range(0,tweet_r.shape[0]):    
    # Seleciona e limpa o tweet
    tweet_raw = str(tweet_r.iloc[i])
    tweet = cleanup(tweet_raw.lower())
    dados_r = dados_r + tweet

#Dados irrelevantes
dados_i = ''
for i in range(0,tweet_i.shape[0]):    
    # Seleciona e limpa o tweet
    tweet_raw = str(tweet_i.iloc[i])
    tweet = cleanup(tweet_raw.lower())
    dados_i = dados_i + tweet

In [65]:
#Serie de dados totais
serie_dado = pd.Series(dados)

#Serie de dados dos irrelevantes
serie_dado_i = pd.Series(dados_i.split())
i_relativo = serie_dado_i.value_counts(True) 

#Serie de dados dos relevantes
serie_dado_r = pd.Series(dados_r.split())
r_relativo = serie_dado_r.value_counts(True) 

In [73]:
#Classificador
for i in range(0,tweet_t.shape[0]):
    #Declarando variáveis
    P_D_dado_R = 1
    P_D_dado_I = 1
    P_R_dado_D = 0
    P_I_dado_D = 0    
    #Seleciona e limpa e separa o tweet
    tweet_raw = str(tweet_t.iloc[i])
    tweet = cleanup(tweet_raw.lower())
    tweet = tweet.split()
    #Calcula as probabilidades de ser:
    #-->Relevante
    for i in tweet:
        if i in r_relativo:
            P_D_dado_R = P_D_dado_R * r_relativo[i] 
        else:
            None
        #Resultado
        P_R_dado_D = P_D_dado_R / R
    #-->Irrelevante
    for i in tweet:
        if i in i_relativo:
            P_D_dado_I = P_D_dado_I * i_relativo[i]
        else:
            None
        #Resultado
        P_I_dado_D = P_D_dado_I / I

    #Classificando
    if (P_R_dado_D > P_I_dado_D):
        test.loc[test.Teste == tweet_raw ,'ClassByRel'] = 'relevante'
    else:
        test.loc[test.Teste == tweet_raw,'ClassByRel'] = 'irrelevante'
    

In [ ]:
    #-->Irrelevante
    for i in tweet:
        if not i in dados_r:
            P_D_dado_I = P_D_dado_I 
        else:
            P_D_dado_I = P_D_dado_I * r_relativo[i]
        #Resultado
        P_I_dado_D = P_D_dado_I / I

        #Classificando
        if (P_R_dado_D > P_I_dado_D):
            test.loc[test.Teste == tweet_raw ,'ClassByRel'] = 'relevante'
        else:
            test.loc[test.Teste == tweet_raw,'ClassByRel'] = 'irrelevante'

In [74]:
test.head(5)

,Teste,Relevância,% de 1,ClassByRel
0,entenda como ficará a venda de eletrodoméstico...,irrelevante,48.292683,relevante
1,o novo #playstation 5 está previsto para ser l...,irrelevante,NaN,irrelevante
2,"@jose_simao vem cá, o preço do novo playstatio...",relevante,NaN,irrelevante
3,@yasminxlima me dá um playstation 5,relevante,NaN,relevante
4,propaganda do playstation 5 aparece em partida...,irrelevante,NaN,irrelevante


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [76]:
pd.crosstab(test.ClassByRel, test.Relevância, normalize='columns')

Relevância,irrelevante,relevante
ClassByRel,,
irrelevante,0.490566,0.707071
relevante,0.509434,0.292929


___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**